## Project: Analysis and Classification of customer churn in telecommunication companies

### Author: Munezero Mihigo

### Date: 13 November 2021

## Project Objective

### The objective of the task is to predict whether the customer will churn or not.

## General Knowledge

### What is a customer churn?
 customer churn occurs when an existing customer, user, player, subscriber or any kind of return client stops doing business or ends the relationship with a company.

### Types of churn

#### Contractual churn:
 Contractual churn occurs when a customer is under contract for a service and decides to cancel their service. <br>Example: Cable TV, SaaS products (Software as a Service e.g. Dropbox).

#### Voluntary churn:
 Voluntary churn occurs when a user voluntarily cancels a service and includes prepaid cell phones, streaming subscriptions.

#### Non-contractual churn:
 Non-contractual churn ocurs when a customer is not under contract for a service and includes customer loyality at a retail location or online browsing.

#### Involuntary churn:
 Involuntary churn occurs when a churn occurs not at the request of the customer. For example: credit card expiration, utilities being shut off by the provider. <br>Most likely, you as a customer have cancelled a service for a variety of reasons including lack of usage, poor service or better price.

In [11]:
# Import libraries
import numpy as np
import pandas as pd
from scipy.stats.mstats import gmean
import seaborn as sns
import matplotlib.pyplot as plt